In [1]:
import numpy as np

# Importo mi condicion inicial a priori
condition_initial = np.array([
    [0.25],
    [0.75]
])

# Mi matriz de transicion
transicion = np.array([
    [0.7, 0.3],
    [0.1, 0.9]
])

# Mi matriz de emision
emision = np.array([
    [0.9, 0.1],
    [0.2, 0.8]
])

In [2]:
class Viterbi(object):
    def __init__(self, condition_initial, transicion, emision):
        self.N = condition_initial.shape[0]
        self.condition_initial = condition_initial.copy()
        self.transicion = transicion.copy()
        self.emision = emision.copy()
        
        #Validaciones en las dimensiones de datos de entrada
        assert self.condition_initial.shape == (self.N, 1)
        assert self.transicion.shape == (self.N, self.N)
        assert self.emision.shape[0] == self.N
 
    def Obs(self, obs):
        return self.emision[:, obs, None]
    
    def standardization(self, arrayx):
        return arrayx / sum(arrayx)
 
    def predict(self, obs):
        """
            Se crea el enrejado inicial con numero de filas = # estados ocultos
            y numero de columnas = # de observaciones provistas
        """
        size_initial =  (self.N, len(obs))
        trellis = np.zeros(size_initial)
        backpt = np.ones(size_initial, 'int32') * -1
        
        trellis[:, 0] = self.standardization(
            np.squeeze(self.condition_initial * self.Obs(obs[0]))
        )
        
        for t in range(1, len(obs)):
            trellis[:, t] = self.standardization(
                (trellis[:, t-1, None].dot(self.Obs(obs[t]).T) * self.transicion).max(0)
            )
            backpt[:, t] = (np.tile(trellis[:, t-1, None], [1, self.N]) * self.transicion).argmax(0)
        
        print("=================================== Enrejado")
        print(trellis)
        print("===="*12)
        tokens = [trellis[:, -1].argmax()]
        for i in range(len(obs)-1, 0, -1):
            tokens.append(backpt[tokens[-1], i])
        return tokens[::-1]

In [3]:
# Instanciamos nuestro algoritmo
algoritmo = Viterbi(condition_initial, transicion, emision)

In [4]:
print("#############TESTEO DE ALGORITMO #################")
print("\n Durante 5 días consecutivos se vio un paraguas")
print("Estados mas probables", algoritmo.predict([1, 1, 1, 1, 1]))
print("\n")

print("\n Durante 5 días consecutivos NO se vio un paraguas")
print("Estados mas probables", algoritmo.predict([0, 0, 0, 0, 0]))
print("\n")

print("\n Solo el Lunes Martes y Jueves se vio paraguas")
print("Estados mas probables", algoritmo.predict([1, 1, 0, 1, 0]))
print("\n")

print("\n Solo Miercoles y Viernes se vio paraguas")
print("Estados mas probables", algoritmo.predict([0, 0, 1, 0, 1]))
print("\n")

print("\n Solo el Jueves se vio paraguas")
print("Estados mas probables", algoritmo.predict([0, 0, 0, 1, 0]))
print("\n")

#############TESTEO DE ALGORITMO #################

 Durante 5 días consecutivos se vio un paraguas
=================================== Enrejado
[[0.04       0.01369863 0.01369863 0.01369863 0.01369863]
 [0.96       0.98630137 0.98630137 0.98630137 0.98630137]]
Estados mas probables [1, 1, 1, 1, 1]



 Durante 5 días consecutivos NO se vio un paraguas
=================================== Enrejado
[[0.6        0.84       0.91304348 0.91304348 0.91304348]
 [0.4        0.16       0.08695652 0.08695652 0.08695652]]
Estados mas probables [0, 0, 0, 0, 0]



 Solo el Lunes Martes y Jueves se vio paraguas
=================================== Enrejado
[[0.04       0.01369863 0.33333333 0.04635762 0.33333333]
 [0.96       0.98630137 0.66666667 0.95364238 0.66666667]]
Estados mas probables [1, 1, 1, 1, 1]



 Solo Miercoles y Viernes se vio paraguas
=================================== Enrejado
[[0.6        0.84       0.22580645 0.50515464 0.09028692]
 [0.4        0.16       0.77419355 0.49484536 0.